# How to Use Trained Model: A Guide

This notebook provides a step-by-step guide on how to load and use a pre-trained model for downstream tasks. Whether you're looking to fine-tune the model for a specific task, perform inference, or explore the features learned by the model, this notebook will help you get started.

## Objectives:
- **Load a Pre-Trained Model:** Learn how to load a model that has been trained using the provided scripts.
- **Prepare Data for Evaluation and Visualization:** Set up your dataset specifically for assessing the model’s performance and visualizing its learned features.
- **Evaluate Performance:** Assess the model’s performance on your dataset using standard metrics.
- **Visualize Model Features:** Visualize the features extracted by the model.
- **Extract Features Using the Model's Backbone:** Learn how to extract and utilize the backbone of the pre-trained model for feature extraction.
- **Implement a Toy Example of Supervised Classification:** See a practical demonstration of how to use the backbone of the pre-trained model for a simple supervised classification task.
- **Customize for Your Needs:** Extend this notebook to suit your specific requirements.

## Prerequisites:
- A trained model saved using the training scripts provided in this project.

## Contents:
1. [Loading the Pre-Trained Model](#loading-the-pre-trained-model)
2. [Preparing Data for Evaluation and Visualization](#preparing-data-for-evaluation-and-visualization)
3. [Evaluating Performance](#evaluating-performance)
4. [Visualizing Model Features](#visualizing-model-features)
5. [Extracting the Backbone for Downstream Tasks](#extracting-the-backbone-for-downstream-tasks)
6. [Toy Example: Supervised Classification with the Backbone](#toy-example-supervised-classification-with-the-backbone)
7. [Customizing for Your Needs](#customizing-for-your-needs)


<a id="loading-the-pre-trained-model"></a>
## 1. Loading the Pre-Trained Model


In [1]:
import torch
from modules.network import Network

# Define the model parameters (ensure these match the parameters used during training)
model_path = 'models/cifar10/model_epoch_1000.pth'
backbone = 'ResNet34'
feature_num = 128
hidden_dim = 128

# Define the device to be used for computation (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# For Apple Silicon users: Check if 'mps' backend is available and use it if possible
if device == torch.device('cpu') and hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')

# Initialize the model architecture
model = Network(backbone=backbone, feature_num=feature_num, hidden_dim=hidden_dim)

# Load the trained model weights
model.load_state_dict(torch.load(model_path))

# Move the model to the appropriate device
model.to(device)

print("Model loaded successfully!")


Model loaded successfully!


<a id="preparing-data-for-evaluation-and-visualization"></a> 
## 2. Preparing Data for Evaluation and Visualization

In [2]:
from dataset import get_data_loader

# Define the dataset and other parameters (ensure these match what was used during training)
config = {
    'dataset': 'cifar10',
    'batch_size': 128,
    's': 1.0,            # Color jitter strength (default)
    'blur': False,       # Whether to apply Gaussian blur (default)
    'customize_datasets': False  # Whether to use custom datasets
}

# Load the train, test data loaders
train_loader, test_loader, _ = get_data_loader(config)

print("Data loaders prepared successfully!")


Files already downloaded and verified
Files already downloaded and verified
Data loaders prepared successfully!


<a id="measuring-metrics"></a>
## 3. Measuring Metrics

In [3]:
from utils.metrics import evaluate

# Perform evaluation on the test data and measure the metrics
nmi_backbone, ari_backbone, acc_backbone, nmi_feature, ari_feature, acc_feature = evaluate(model, test_loader, device=device)

# Print the results
print(f"Backbone NMI: {nmi_backbone:.4f}, ARI: {ari_backbone:.4f}, ACC: {acc_backbone:.4f}")
print(f"Feature NMI: {nmi_feature:.4f}, ARI: {ari_feature:.4f}, ACC: {acc_feature:.4f}")


Backbone NMI: 0.7249, ARI: 0.6199, ACC: 0.7992
Feature NMI: 0.7109, ARI: 0.6243, ACC: 0.7876


<a id="visualizing-model-features"></a>
## 4. Visualizing Model Features

We'll display the t-SNE plots for both the backbone and feature embeddings.

In [4]:
from utils.visualization import visualize_embeddings

# Visualize embeddings using t-SNE
# The first image corresponds to the backbone, and the second image corresponds to the feature predictor.
visualize_embeddings(model, test_loader, device=device, epoch=1000, name='cifar10')

print("t-SNE visualizations generated!")


t-SNE visualizations generated!


<a id="extracting-the-backbone-for-downstream-tasks"></a> 
## 5. Extracting the Backbone for Downstream Tasks

Let's explore how to access and use the backbone from the pre-trained model. The backbone, which is typically a feature extractor like ResNet, can be leveraged for various downstream tasks such as classification, detection, or segmentation.

In [5]:
# Access the backbone from the model
backbone = model.resnet

# Example: Extract features using the backbone
with torch.no_grad():
    for (x, _, _) in train_loader:
        x = x.to(device)
        
        # Pass the input through the backbone to extract features
        features = backbone(x)

        # Display the shape of the extracted features
        print(f"Extracted features shape: {features.shape}")
        
        # Break after the first batch to avoid unnecessary computations
        break


Extracted features shape: torch.Size([128, 512])


<a id="toy-example-supervised-classification-with-the-backbone"></a>
## 6. Toy Example: Supervised Classification with the Backbone

In this section, we will utilize the backbone of the pre-trained model to perform supervised classification on the current dataset. This is a toy example designed to demonstrate the basic steps involved: extracting features using the backbone, adding a simple classification head, training the model on the dataset, and evaluating its accuracy.

This example uses a small number of epochs and a simple dataset to illustrate the process, making it easy to follow along and adapt for more complex scenarios.

### 6.1 Preparing Data
Define the transformations and load the CIFAR-10 dataset for training and testing.

In [6]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define the transformations for training and testing
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()
])

# Load datasets with the defined transforms
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

# Define the data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


### 6.2 Defining the Model
Next, we define a toy classifier model that uses the backbone for feature extraction, followed by a fully connected layer for classification.

The backbone of a pre-trained model is typically used to extract meaningful features from the data. By removing the final layers, we can repurpose these features for a new task.

In [7]:
from torch import nn

# Define a simple classifier using the backbone
class ToyClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super(ToyClassifier, self).__init__()
        self.backbone = backbone
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        with torch.no_grad():  # Freeze the backbone
            features = self.backbone(x)
        out = self.fc(features)
        return out

### 6.3 Setting Up Training
Set up the loss function, optimizer, and the training loop.

In [8]:
import torch.optim as optim
from tqdm import tqdm

# Assuming you have a dataset and dataloaders ready
num_classes = 10  # Set this to the number of classes in your dataset
model = ToyClassifier(backbone, num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop for 5 epochs
epochs = 5
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Iterate over the training data
    for x, y in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
        x, y = x.to(device), y.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(x)
        
        # Calculate the loss
        loss = criterion(outputs, y)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Update running loss and accuracy
        running_loss += loss.item() * x.size(0)
        predictions = outputs.argmax(1)
        total += y.size(0)
        correct += predictions.eq(y).sum().item()
    
    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = 100. * correct / total
    
    # Print epoch statistics
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%")


Epoch 1/5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:59<00:00, 26.10it/s]


Epoch [1/5], Loss: 0.5063, Accuracy: 84.59%


Epoch 2/5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [01:00<00:00, 25.99it/s]


Epoch [2/5], Loss: 0.4125, Accuracy: 85.94%


Epoch 3/5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:59<00:00, 26.18it/s]


Epoch [3/5], Loss: 0.4016, Accuracy: 86.26%


Epoch 4/5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:59<00:00, 26.18it/s]


Epoch [4/5], Loss: 0.3957, Accuracy: 86.58%


Epoch 5/5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1563/1563 [00:59<00:00, 26.15it/s]

Epoch [5/5], Loss: 0.3940, Accuracy: 86.64%


### 6.4 Evaluating the Model

In [9]:
# Set the model to evaluation mode
model.eval()

# Initialize counters for correct predictions and total samples
correct = 0
total = 0

# Disable gradient calculation for evaluation
with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Evaluating"):
        # Move input data and labels to the appropriate device
        x, y = x.to(device), y.to(device)
        
        # Get model predictions
        outputs = model(x)
        
        # Get the predicted class by taking the index with the highest score
        predictions = outputs.argmax(1)
        
        # Update total count and correct predictions
        total += y.size(0)
        correct += predictions.eq(y).sum().item()

# Calculate accuracy
test_accuracy = 100. * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 313/313 [00:11<00:00, 27.30it/s]

Test Accuracy: 91.32%


<a id="customizing-for-your-needs"></a>
## 7. Customizing for Your Needs

This notebook just provides a starting point. You can customize it by:
- Implementing additional downstream tasks, such as fine-tuning the model.
- Integrating the model into a larger pipeline.

We hope this notebook helps you get started. Feel free to modify it to suit your specific needs!